Рассчёт различных числовых характеристик (метрик) полиномиальных идеалов, для последующей оценки потенциального время вычисления базиса Грёбнера

In [ ]:
# импорт библиотек
from re import split # для обработки строкового представления полиномов
import os
import json
import pandas as pd

Поключение Диска (с тестами)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
VERY_QUICK = ['quadfor2', 'sparse5', 'hunecke', 'solotarev', 'chandra4', 'quadgrid', 'lorentz', 'liu', 'hemmecke', 'boon', 'chandra5', 'caprasse', 'issac97', 'hcyclic5', 'redcyc5', 'cyclic5', 'extcyc4', 'chemequ', 'uteshev_bikker', 'chandra6', 'geneig']
QUICK = ['chemequs', 'vermeer', 'camera1s', 'reimer4', 'redeco7', 'tangents', 'cassou', 'butcher', 'eco7', 'cohn2', 'dessin1', 'des18_3', 'hcyclic6', 'noon5', 'katsura6', 'cyclic6', 'butcher8', 'redcyc6', 'cpdm5', 'extcyc5']
MEDIUM = ['noon6', 'reimer5', 'kotsireas']
I66 = ['assur44', 'aubry2', 'augot', 'benchmark_D1', 'benchmark_i1', 'boon', 'butcher', 'butcher8', 'camera1s', 'caprasse', 'cassou', 'chandra4', 'chandra5', 'chandra6', 'chemequ', 'chemequs', 'chemkin', 'cohn2', 'cohn3', 'comb3000', 'comb3000s', 'conform1', 'cpdm5', 'cyclic5', 'cyclic6', 'cyclic7', 'd1', 'des18_3', 'des22_24', 'dessin1', 'dessin2', 'dl', 'eco10', 'eco7', 'eco8', 'eco9', 'extcyc4', 'extcyc5', 'f633', 'f744', 'f966', 'fabrice24', 'filter9', 'geneig', 'hairer1', 'hairer2', 'hairer3', 'hcyclic5', 'hcyclic6', 'hcyclic7', 'heart', 'hemmecke', 'hf744', 'hietarinta1', 'hunecke', 'i1', 'ilias_k_3', 'ilias12', 'ilias13', 'issac97', 'jcf26', 'rose', 'solotarev', 'speer', 'tangents', 'uteshev_bikker']

Примеры идеалов

In [ ]:
# переменные для волчков
osc_vars = ['x1','x2','x3','x4','x5','x6','x7','y1','y2','y3','y4','y5','y6','y7','T']

In [ ]:
# для нелинейного волчка по 5 точкам
def get_osc_5():
   return [
        '10*x1 - T*y1 - T',
        '-10*x1 + 10*x2 - T*y1 - T*y2',
        '-10*x2 + 10*x3 - T*y2 - T*y3',
        '-10*x3 + 10*x4 - T*y3 - T*y4',
        '-10*x4 - T*y4 - T',
        '40*y1 - 40 + T*x1**3',
        '-40*y1 + 40*y2 + T*x1**3 + T*x2**3 + 3*T*x1**2*x2 + 3*T*x1*x2**2',
        '-40*y2 + 40*y3 + T*x2**3 + T*x3**3 + 3*T*x2**2*x3 + 3*T*x2*x3**2',
        '-40*y3 + 40*y4 + T*x3**3 + T*x4**3 + 3*T*x3**2*x4 + 3*T*x3*x4**2',
        '-40*y4 + 40 + T*x4**3'
    ]

In [ ]:
# для нелинейного волчка по 7 точкам
def get_osc_7():
   return [
        '14*x1 - T*y1 - T',
        '-14*x1 + 14*x2 - T*y1 - T*y2',
        '-14*x2 + 14*x3 - T*y2 - T*y3',
        '-14*x3 + 14*x4 - T*y3 - T*y4',
        '-14*x4 + 14*x5 - T*y4 - T*y5',
        '-14*x5 + 14*x6 - T*y5 - T*y6',
        '-14*x6 - T*y6 - T',
        '56*y1 + T*x1**3 - 56',
        '-56*y1 + 56*y2 + T*x1**3 + 3*T*x1**2*x2 + 3*T*x1*x2**2 + T*x2**3',
        '-56*y2 + 56*y3 + T*x2**3 + 3*T*x2**2*x3 + 3*T*x2*x2**3 + T*x3**3',
        '-56*y3 + 56*y4 + T*x3**3 + 3*T*x3**2*x4 + 3*T*x3*x2**4 + T*x4**3',
        '-56*y4 + 56*y5 + T*x4**3 + 3*T*x4**2*x5 + 3*T*x4*x2**5 + T*x5**3',
        '-56*y5 + 56*y6 + T*x5**3 + 3*T*x5**2*x6 + 3*T*x5*x2**6 + T*x6**3',
        '-56*y6 + T*x6**3 + 56'
    ]

Код для формирования и вычисления метрик

In [ ]:
def init_metrics():
  metrics = {
    'Имя теста' : '',
    'Средний коэф.' : -1,
    'Максимальный коэф.' : -1,
    'Сумма коэф.' : -1,
    'Средняя степень по моному' : -1,
    'Средняя степень по литеру' : -1,
    'Максимальная степень' : -1,
    'Сумма степеней' : -1,
    'Кол. мономов' : -1,
    'Кол. полиномов' : -1,
    'Кол. литеров' : -1,
    'Среднее мономов на полином' : -1,
    'Среднее кол. литеров в мономе' : -1,
    'Максимальная кол. литеров в мономе' : -1,
    'Сумма степеней НОК' : -1,
    'Сумма остатков от НОК (по всем мономам)' : -1,
    'Сумма остатков от НОК (по старшим мономам)' : -1,
  }
  return metrics

In [ ]:
def metrics_to_str(metrics):
  str_metrics = dict()
  for key in metrics.keys():
    str_metrics[key] = str(metrics[key]).replace('.', ',')
  return str_metrics

In [ ]:
def calc_metrics(vars, polynoms):
  metrics = init_metrics()
  metrics['Кол. полиномов'] = len(polynoms)
  metrics['Кол. мономов'] = 0
  metrics['Кол. литеров'] = 0
  metrics['Сумма коэф.'] = 0
  metrics['Сумма степеней'] = 0
  var_degs = [0] * len(vars)
  lead_degs = []
  for poly in polynoms:
    poly = poly.replace(" ", "")
    if poly.find('(') != -1 and poly.find(')') == len(poly) - 1:
      poly = poly[poly.find('(') + 1 : -1]
    metrics['Кол. мономов'] += poly[1:].count('+') + poly[1:].count('-') + 1
    monoms = split('\+|-', poly)
    all_degs = []
    for monom in monoms:
      # коэффициенты
      poses = monom.split('*')
      if not (poses[0] in vars) and poses[0] != '':
        coef = abs(int(poses[0]))
        metrics['Сумма коэф.'] += coef
        if coef > metrics['Максимальный коэф.']:
          metrics['Максимальный коэф.'] = coef
      # литералы
      literal_count = 0
      for i in range(len(vars)):
        literal_count += monom.count(vars[i])
        v_id = monom.find(vars[i])
        if v_id != -1:
          start = v_id + len(vars[i])
          if len(monom) > start + 2 and monom[start : start + 2] == '**':
            end = len(monom) if (monom.find('*', start + 2) == -1) else monom.find('*', start + 2)
            end = min(end, len(monom) if (monom.find('+', start + 2) == -1) else monom.find('+', start + 2))
            end = min(end, len(monom) if (monom.find(' ', start + 2) == -1) else monom.find(' ', start + 2))
            var_degs[i] = max(var_degs[i], int(monom[start + 2 : end]))
          else:
            var_degs[i] = max(var_degs[i], 1)
      if literal_count > metrics['Максимальная кол. литеров в мономе']:
        metrics['Максимальная кол. литеров в мономе'] = literal_count
      metrics['Кол. литеров'] += literal_count #poly.count(v)
      # степени
      d_pos = monom.find('**')
      all_degs.append(0)
      while d_pos != -1:
        next = monom.find('*', d_pos+2)
        if next == -1:
          next = len(monom)
        deg = int(monom[d_pos+2:next])
        metrics['Сумма степеней'] += deg - 1
        metrics['Средняя степень по моному'] += deg - 1
        if deg > metrics['Максимальная степень']:
          metrics['Максимальная степень'] = deg
        all_degs[-1] = all_degs[-1] + deg
        d_pos = monom.find('**',next)
      metrics['Средняя степень по моному'] += literal_count
    lead_degs.append(max(all_degs))

  metrics['Среднее мономов на полином'] = metrics['Кол. мономов'] / metrics['Кол. полиномов']
  metrics['Среднее кол. литеров в мономе'] = metrics['Кол. литеров'] / metrics['Кол. мономов']
  metrics['Средний коэф.'] = metrics['Сумма коэф.'] / metrics['Кол. мономов']
  metrics['Сумма степеней'] += metrics['Кол. литеров']
  metrics['Сумма степеней НОК'] = sum(var_degs)
  metrics['Средняя степень по моному'] = metrics['Средняя степень по моному'] / metrics['Кол. мономов']
  metrics['Средняя степень по литеру'] = metrics['Сумма степеней'] / metrics['Кол. литеров']
  metrics['Сумма остатков от НОК (по всем мономам)'] = metrics['Кол. мономов'] * metrics['Сумма степеней НОК'] - metrics['Сумма степеней']
  metrics['Сумма остатков от НОК (по старшим мономам)'] = 0
  for lead in lead_degs:
    metrics['Сумма остатков от НОК (по старшим мономам)'] += metrics['Сумма степеней НОК'] - lead

  return metrics

In [ ]:
# путь до тестов
path = 'drive/MyDrive/GinvTests'
# путь рассчёт метрик для теста из диска
def calc_test_metric(test_name):
  data = json.load(open(path + '/'+ test_name + '.json'))
  vars = data['variables']
  equations = data['equations']
  eqs = []
  for eq in equations:
    eqs.append(eq.replace('^', '**'))
  metrics = calc_metrics(vars, eqs)
  metrics['Имя теста'] = test_name
  return metrics_to_str(metrics)

In [ ]:
print(calc_metrics(osc_vars, get_osc_5()))

{'Имя теста': '', 'Средний коэф.': 11.857142857142858, 'Максимальный коэф.': 40, 'Сумма коэф.': 498, 'Средняя степень по моному': 2.119047619047619, 'Средняя степень по литеру': 1.3235294117647058, 'Максимальная степень': 3, 'Сумма степеней': 90, 'Кол. мономов': 42, 'Кол. полиномов': 10, 'Кол. литеров': 68, 'Среднее мономов на полином': 4.2, 'Среднее кол. литеров в мономе': 1.619047619047619, 'Максимальная кол. литеров в мономе': 3, 'Сумма степеней НОК': 17, 'Сумма остатков от НОК (по всем мономам)': 624, 'Сумма остатков от НОК (по старшим мономам)': 155}


Вывод результатов

In [ ]:
# Cброс результатов
results = []

In [ ]:
# рассчёт
print('Very quick')
for test in VERY_QUICK:
  print('...parsing',test)
  results.append(calc_test_metric(test))

print('Quick')
for test in QUICK:
  print('...parsing',test)
  results.append(calc_test_metric(test))

print(results)

Very quick
...parsing quadfor2
...parsing sparse5
...parsing hunecke
...parsing solotarev
...parsing chandra4
...parsing quadgrid
...parsing lorentz
...parsing liu
...parsing hemmecke
...parsing boon
...parsing chandra5
...parsing caprasse
...parsing issac97
...parsing hcyclic5
...parsing redcyc5
...parsing cyclic5
...parsing extcyc4
...parsing chemequ
...parsing uteshev_bikker
...parsing chandra6
...parsing geneig
Quick
...parsing chemequs
...parsing vermeer
...parsing camera1s
...parsing reimer4
...parsing redeco7
...parsing tangents
...parsing cassou
...parsing butcher
...parsing eco7
...parsing cohn2
...parsing dessin1
...parsing des18_3
...parsing hcyclic6
...parsing noon5
...parsing katsura6
...parsing cyclic6
...parsing butcher8
...parsing redcyc6
...parsing cpdm5
...parsing extcyc5
[{'Имя теста': 'quadfor2', 'Средний коэф.': '0,9', 'Максимальный коэф.': '3', 'Сумма коэф.': '9', 'Средняя степень по моному': '1,9', 'Средняя степень по литеру': '1,4285714285714286', 'Максимальная 

In [ ]:
# вывод
# перевод в строки
for row in results:
  for cell in row:
    cell = str(cell).replace('.',',')

df = pd.DataFrame(results)
df.to_csv('metrics.csv', sep=';', encoding = 'utf-8-sig"')